In [1]:
import os
from dotenv import load_dotenv

In [2]:
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT="pr-large-miracle-98"
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [3]:
load_dotenv(override=True)

True

In [4]:
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

In [6]:
from langsmith import traceable

In [7]:
@traceable()
def retrieve_documents(question:str):
    return retriever.invoke(question)

@traceable()
def generate_response(question:str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [{"role":"system", "content":RAG_SYSTEM_PROMPT},
                {"role":"user", "content": f"Context: {formatted_docs} \n\n Question: {question}"}]

    return call_openai(messages)

@traceable()
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0) ->str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )

@traceable
def langsmith_rag(question:str):
    documents = retrieve_documents(question)
    response = generate_response(question,documents)
    return response.choices[0].message.content


In [8]:
question = "How do I add Metadata to run with @traceable"

ai_answer = langsmith_rag(question)

print(ai_answer)

To add metadata to a run using the `@traceable` decorator, you can include the `metadata` parameter within the decorator. For example, you would use `@traceable(metadata={"key": "value"})` to specify the key-value pairs you want to associate with the run. This allows you to log additional information, such as the LLM used or other relevant attributes.
